In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import datetime

## Download and read raw data from Github

#### Define paths

In [2]:
# Root of project
gpl_covid_path = Path(os.getcwd()).parent.parent.parent

dir_data = gpl_covid_path / 'data'

# Administrative unit names
path_adm = dir_data / 'interim' / 'adm' / 'adm.csv'

# Template for processed dataset (output of this notebook)
path_template = dir_data / 'processed' / '[country]_processed.csv'

dir_italy_raw = gpl_covid_path / 'data' / 'raw' / 'italy'
dir_italy_interim = gpl_covid_path / 'data' / 'interim' / 'italy'
dir_processed = gpl_covid_path / 'data' / 'processed'

today = datetime.datetime.now().strftime("%Y%m%d")

# Inputs
# CSV form of policies Google sheet
path_italy_policies = dir_italy_raw / f'italy_policy_static_{today}.csv'
url_cases_by_province = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv"
url_cases_by_region = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"

# Outputs
## Intermediate outputs
path_italy_raw_province = dir_italy_raw / 'italy-cases-by-province.csv'
path_italy_raw_region = dir_italy_raw / 'italy-cases-by-region.csv'
path_italy_interim_province = dir_italy_interim / 'italy-cases-by-province.csv'
path_italy_interim_region = dir_italy_interim / 'italy-cases-by-region.csv'

## Final outputs
path_processed_region = dir_processed / 'adm1' / 'ITA_processed.csv'
path_processed_province = dir_processed / 'adm2' / 'ITA_processed.csv'

#### Read inputs

In [3]:
# Administrative unit names
adm_df = pd.read_csv(path_adm)

# Columns in template (i.e. columns allowed in output)
template_cols = set(pd.read_csv(path_template).columns)

# Italy-specific data
cases_by_province = pd.read_csv(url_cases_by_province)
cases_by_region = pd.read_csv(url_cases_by_region)
policies_full = pd.read_csv(path_italy_policies)

##### Save raw case data from URL to project folder

In [4]:
cases_by_province.head()

,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi
0,2020-02-24 18:00:00,ITA,13,Abruzzo,69,Chieti,CH,42.351032,14.167546,0
1,2020-02-24 18:00:00,ITA,13,Abruzzo,66,L'Aquila,AQ,42.351222,13.398438,0
2,2020-02-24 18:00:00,ITA,13,Abruzzo,68,Pescara,PE,42.464584,14.213648,0
3,2020-02-24 18:00:00,ITA,13,Abruzzo,67,Teramo,TE,42.658918,13.704400,0
4,2020-02-24 18:00:00,ITA,13,Abruzzo,979,In fase di definizione/aggiornamento,NaN,0.000000,0.000000,0


In [5]:
cases_by_region.head()

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
0,2020-02-24 18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,0,0,0,0,0,5
1,2020-02-24 18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,0,0,0,0,0,0
2,2020-02-24 18:00:00,ITA,4,P.A. Bolzano,46.499335,11.356624,0,0,0,0,0,0,0,0,0,1
3,2020-02-24 18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,0,0,0,0,0,1
4,2020-02-24 18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,0,0,0,0,0,10


In [6]:
cases_by_province.to_csv(path_italy_raw_province, index=False)
cases_by_region.to_csv(path_italy_raw_region, index=False)

## Translate and clean health data

###### Settings

In [7]:
cumulative_prefix = 'cum_'
imputed_suffix = '_imputed'
popweighted_suffix = '_popwt'
optional_suffix = '_opt'

### Translate field names from Italian to project naming scheme

In [8]:
# Column names based on table descriptions here: https://github.com/pcm-dpc/COVID-19
replace_dict = {
    'data':'date',
    'lat':'lat',
    'long':'lon',
    'stato':'adm0_name',
    'denominazione_regione':'adm1_name',
    'denominazione_provincia':'adm2_name',
    'codice_regione':'adm1_id',
    'codice_provincia':'adm2_id',
    'sigla_provincia':'province_abbrev',
    'totale_attualmente_positivi':'active_cases',
    'nuovi_attualmente_positivi':'active_cases_new',
    'totale_casi':cumulative_prefix + 'confirmed_cases',
    'ricoverati_con_sintomi':cumulative_prefix + 'hospitalized_symptom',
    'terapia_intensiva':cumulative_prefix + 'intensive_care',
    'totale_ospedalizzati':cumulative_prefix + 'hospitalized',
    'isolamento_domiciliare':cumulative_prefix + 'home_confinement',
    'dimessi_guariti': cumulative_prefix + 'recoveries',
    'deceduti': cumulative_prefix + 'deaths',
    'totale_casi': cumulative_prefix + 'confirmed_cases',
    'tamponi':cumulative_prefix + 'tests',
}

cases_by_province = cases_by_province.rename(columns=replace_dict)
cases_by_region = cases_by_region.rename(columns=replace_dict)

Clean date column

In [9]:
def extract_date_from_datetime(dates):
    return pd.to_datetime(dates.str[:10])

cases_by_province['date'] = extract_date_from_datetime(cases_by_province['date'])
cases_by_region['date'] = extract_date_from_datetime(cases_by_region['date'])

Clean lat-lon coordinates

In [10]:
cases_by_province.loc[:,['lat','lon']] = cases_by_province.loc[:,['lat','lon']].replace(0, np.nan)
assert cases_by_region['lat'].isnull().sum() == 0
assert cases_by_region['lon'].isnull().sum() == 0

Clean unknown province names

In [11]:
# "In fase di definizione/aggiornamento" translates to "Being defined / updated"
cases_by_province['adm2_name'] = cases_by_province['adm2_name'].replace(
    'In fase di definizione/aggiornamento', 'Unknown')

Drop unnecessary column

In [12]:
cases_by_province = cases_by_province.drop(columns=['province_abbrev'])

### Impute values in cases where cumulative counts rise and then fall

In [13]:
def convert_non_monotonic_to_nan(array):
    """Converts a numpy array to a monotonically increasing one.
    Args:
        array (numpy.ndarray [N,]): input array
    Returns:
        numpy.ndarray [N,]: some values marked as missing, all non-missing
            values should be monotonically increasing
    Usage:
        >>> convert_non_monotonic_to_nan(np.array([0, 0, 5, 3, 4, 6, 3, 7, 6, 7, 8]))
        np.array([ 0.,  0., np.nan,  3., np.nan, np.nan,  3., np.nan,  6.,  7.,  8.])
    """
    keep = np.arange(0, len(array))
    is_monotonic = False
    while not is_monotonic:
        is_monotonic_array = np.hstack((
            array[keep][1:] >= array[keep][:-1], np.array(True)))
        is_monotonic = is_monotonic_array.all()
        keep = keep[is_monotonic_array]
    out_array = np.full_like(array.astype(np.float), np.nan)
    out_array[keep] = array[keep]
    return out_array

def log_interpolate(array):
    """Interpolates assuming log growth.
    Args:
        array (numpy.ndarray [N,]): input array with missing values
    Returns:
        numpy.ndarray [N,]: all missing values will be filled
    Usage:
        >>> log_interpolate(np.array([0, np.nan, 2, np.nan, 4, 6, np.nan, 7, 8]))
        np.array([0, 0, 2, 3, 4, 6, 7, 7, 8])
    """
    idx = np.arange(0, len(array))
    log_array = np.log(array.astype(np.float32) + 1e-1)
    interp_array = np.interp(
        x=idx, xp=idx[~np.isnan(array)], fp=log_array[~np.isnan(array)])
    return np.round(np.exp(interp_array)).astype(np.int32)

def impute_cumulative_array(array):
    """Ensures array is cumulative, imputing where necessary
    Args:
        array-like (numpy.ndarray [N,], pandas.Series, etc.): input array with missing values
    Returns:
        numpy.ndarray [N,]: all non-monotonic values will be filled by logarithmic interpolation
    Usage:
        >>> impute_cumulative_array(np.array([0, 0, 5, 3, 4, 6, 3, 7, 6, 7, 8]))
        np.array([0, 0, 2, 3, 4, 6, 7, 7, 8])
    """
    array = np.array(array)
    array = convert_non_monotonic_to_nan(array)
    array = log_interpolate(array)
    return array

def impute_cumulative_df(df, src_col, dst_col, groupby_col):
    """Calculates imputed columns and returns 
    Args:
        df (pandas.DataFrame): input DataFrame with a cumulative column
        src_col (str): name of cumulative column to impute
        dst_col (str): name of imputed cumulative column
        groupby_col (str): name of column containing names of administrative units,
            values should correspond to groups whose values should be accumulating
    Returns:
        pandas.DataFrame: a copy of `df` with a newly imputed column specified by `dst_col`
    Usage:
        >>> impute_cumulative_df(pandas.DataFrame([[0, 'a'], [5, 'b'], [3, 'a'], [2, 'a'], [6, 'b']]), 0, 1)
        pandas.DataFrame([[0, 'a', 0], [5, 'b', 5], [3, 'a', 0], [2, 'a', 2], [6, 'b', 6]], columns=[0, 1, 'imputed'])
    """
    if dst_col not in df.columns:
        df[dst_col] = -1

    for adm_name in df[groupby_col].unique():
        sub = df.loc[df[groupby_col] == adm_name].copy()
        sub[dst_col] = impute_cumulative_array(sub[src_col])
        df.loc[df[groupby_col] == adm_name] = sub
        
    return df

In [14]:
cases_by_province = impute_cumulative_df(cases_by_province, 'cum_confirmed_cases', 'cum_confirmed_cases' + imputed_suffix, 'adm2_id')

cases_by_region_cum_cols = [col for col in cases_by_region.columns if 'cum_' in col]
for src_col in cases_by_region_cum_cols:
    dst_col = src_col + imputed_suffix
    cases_by_region = impute_cumulative_df(cases_by_region, src_col, dst_col, 'adm1_name')

In [15]:
cases_by_province.to_csv(path_italy_interim_province, index=False)
cases_by_region.to_csv(path_italy_interim_region, index=False)

## Merge Health with Policies

In [16]:
policies_full['Date'] = pd.to_datetime(policies_full['Date'])
policies_full['Policy'] = policies_full['Policy'].str.strip()
policies_full['Opt'] = policies_full['Opt'].replace({"Y":1, "N":0})
policies_full['Opt'] = policies_full['Opt'].fillna('N')

policies_full['adm0_name'] = policies_full['adm0_name'].fillna('Italy')
policies_full['adm1_affected'] = policies_full['adm1_affected'].fillna('All')
policies_full['adm2_affected'] = policies_full['adm2_affected'].fillna('All')
policies_full['adm3_affected'] = policies_full['adm3_affected'].fillna('All')

assert len(policies_full[policies_full['adm1_pop_weight_perc_affected'].isnull()]) == 0
assert len(policies_full[policies_full['adm2_pop_weight_perc_affected'].isnull()]) == 0

policies = policies_full[
    ['adm3_affected','adm2_pop_weight_perc_affected','adm2_affected', 
     'adm1_pop_weight_perc_affected','adm1_affected','adm0_name',
     'Date','Policy','Opt']
].drop_duplicates()

In [17]:
# If this fails, implement `testing_regime` as categorical variable
assert policies.groupby('Policy')['Policy'].count()['testing_regime'] == 1

In [18]:
policies['home_isolation_partial'] = False
policies.loc[policies['Policy'] == 'home_isolation_partial', 'home_isolation_partial'] = True

policies.loc[policies['Policy'] == 'home_isolation_partial', 'adm2_pop_weight_perc_affected'] = policies.loc[policies['Policy'] == 'home_isolation_partial', 'adm2_pop_weight_perc_affected'] * 0.5
policies.loc[policies['Policy'] == 'home_isolation_partial', 'adm1_pop_weight_perc_affected'] = policies.loc[policies['Policy'] == 'home_isolation_partial', 'adm1_pop_weight_perc_affected'] * 0.5
policies['Policy'] = policies['Policy'].replace('home_isolation_partial', 'home_isolation')

policies.loc[policies['Opt'] == 1, 'Policy'] = policies.loc[policies['Opt'] == 1, 'Policy'] + optional_suffix

In [19]:
replace_dict = {
    'Lombardy':'Lombardia',
    'Piedmont':'Piemonte',
    'Emilia-Romagna':'Emilia Romagna',
    'Padua':'Padova',
    'Venice':'Venezia',
    'Pesaro and Urbino':'Pesaro e Urbino',
    'Apulia':'Puglia', 
}

In [20]:
adm1_not_found = set(policies['adm1_affected'].unique()) - set(cases_by_region['adm1_name'].unique()) - set(['All']) - set(replace_dict.keys())
adm2_not_found = set(policies['adm2_affected'].unique()) - set(cases_by_province['adm2_name'].unique()) - set(['All']) - set(replace_dict.keys())
print(adm1_not_found, adm2_not_found)
assert len(adm1_not_found) == 0
assert len(adm2_not_found) == 0

set() set()


In [21]:
# Need to translate all regions
policies['adm1_affected'] = policies['adm1_affected'].replace(replace_dict)

# Need to translate all provinces
policies['adm2_affected'] = policies['adm2_affected'].replace(replace_dict)

Interpret `home_isolation_partial` as `home_isolation` with a 1/2 multiplier

In [22]:
policies_by_region = pd.DataFrame(policies.groupby(['Date', 'adm1_affected', 'Policy', 'Opt', 'home_isolation_partial'])['adm1_pop_weight_perc_affected'].sum()).reset_index()
policies_by_province = pd.DataFrame(policies.groupby(['Date', 'adm1_affected', 'adm2_affected', 'Policy', 'Opt', 'home_isolation_partial'])['adm2_pop_weight_perc_affected'].sum()).reset_index()

Assign policy indicators

In [23]:
for policy_name in policies['Policy'].unique():
    cases_by_region[policy_name] = 0
    cases_by_region[policy_name + popweighted_suffix] = 0
    cases_by_province[policy_name] = 0
    cases_by_province[policy_name + popweighted_suffix] = 0

for time, policy, optional, adm, perc_affected, partial in policies_by_region[['Date', 'Policy', 'Opt', 'adm1_affected', 'adm1_pop_weight_perc_affected', 'home_isolation_partial']].to_numpy():
    cases_by_region.loc[
        (cases_by_region['date'] >= time) &
        ((cases_by_region['adm1_name'] == adm) | (adm == 'All')), policy
    ] = 1 if not (policy == 'home_isolation' and partial == True) else 0.5
    
    cases_by_region.loc[
        (cases_by_region['date'] >= time) &
        ((cases_by_region['adm1_name'] == adm) | (adm == 'All')), policy + popweighted_suffix
    ] = perc_affected
    
for time, policy, optional, adm1, adm2, perc_affected, partial in policies_by_province[['Date', 'Policy', 'Opt', 'adm1_affected', 'adm2_affected', 'adm2_pop_weight_perc_affected', 'home_isolation_partial']].to_numpy():
    cases_by_province.loc[
        (cases_by_province['date'] >= time) &
        (
            (cases_by_province['adm2_name'] == adm2) | 
            (adm1 == 'All') | 
            (
                (adm2 == 'All') & (adm1 == cases_by_province['adm1_name'])
            )
        ), policy
    ] = 1 if not (policy == 'home_isolation' and partial == True) else 0.5
    
    cases_by_province.loc[
        (cases_by_province['date'] >= time) &
        (
            (cases_by_province['adm2_name'] == adm2) | 
            (adm1 == 'All') | 
            (
                (adm2 == 'All') & (adm1 == cases_by_province['adm1_name'])
            )
        ), policy + popweighted_suffix
    ] = perc_affected
    
cases_by_region['policies_enacted'] = 0
cases_by_province['policies_enacted'] = 0
for policy_name in policies['Policy'].unique():
    cases_by_region['policies_enacted'] += cases_by_region[policy_name]
    cases_by_province['policies_enacted'] += cases_by_province[policy_name]

In [24]:
cases_by_region['no_gathering_size'] = 0
cases_by_province['no_gathering_size'] = 0

In [25]:
travel_international_popweighted_cols = [col for col in cases_by_region.columns if 'travel_ban_intl' in col and popweighted_suffix in col]
assert len(travel_international_popweighted_cols) > 0
cases_by_region = cases_by_region.drop(columns=travel_international_popweighted_cols)
cases_by_province = cases_by_province.drop(columns=travel_international_popweighted_cols)

cases_by_region = cases_by_region.drop(columns='testing_regime' + popweighted_suffix)
cases_by_province = cases_by_province.drop(columns='testing_regime' + popweighted_suffix)

In [42]:
template = pd.read_csv(path_template)

In [43]:
missing_from_template = (set(cases_by_region.columns) | set(cases_by_province.columns)) - set(template.columns)
missing_from_template

set()

In [34]:
new_template_cols = list(template.columns)[1:]

In [35]:
new_template_cols

['date',
 'adm0_name',
 'adm1_name',
 'adm2_name',
 'adm1_id',
 'adm2_id',
 'lat',
 'lon',
 'cum_confirmed_cases',
 'cum_confirmed_cases_imputed',
 'cum_estimated_cases',
 'cum_estimated_cases_imputed',
 'active_cases',
 'active_cases_imputed',
 'active_cases_new',
 'active_cases_new_imputed',
 'cum_deaths',
 'cum_deaths_imputed',
 'cum_recoveries',
 'cum_recoveries_imputed',
 'cum_home_confinement',
 'cum_home_confinement_imputed',
 'cum_hospitalized',
 'cum_hospitalized_imputed',
 'cum_hospitalized_with_symptoms',
 'cum_hospitalized_symptom_imputed',
 'cum_intensive_care',
 'cum_intensive_care_imputed',
 'cum_tests',
 'cum_tests_imputed',
 'testing_regime',
 'school_closure',
 'school_closure_opt',
 'school_closure_popwt',
 'school_closure_opt_popwt',
 'business_closure',
 'business_closure_opt',
 'business_closure_popwt',
 'business_closure_opt_popwt',
 'no_gathering',
 'no_gathering_opt',
 'no_gathering_popwt',
 'no_gathering_opt_popwt',
 'no_gathering_size',
 'travel_ban_local',
 

In [36]:
cases_by_region.to_csv(path_processed_region, index=False)
cases_by_province.to_csv(path_processed_province, index=False)